In [1]:
# Dependencies and Setup
import pandas as pd

from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Float
from sqlalchemy.orm import Session

from config import username, password, host, database

In [2]:
# File path
global_filepath = '../Resources/mmr_global.csv'

# Read in csv file
mmr_global_df = pd.read_csv(global_filepath, names=['name', 'mmr', 'category', 'location', 'country'])

# Display dataframe
mmr_global_df.head()

,name,mmr,category,location,country
0,Belarus,2,Very Low (<100),53.744 27.947,Belarus
1,Italy,2,Very Low (<100),45.3889 10.7776,Italy
2,Norway,2,Very Low (<100),60.5366 7.4496,Norway
3,Poland,2,Very Low (<100),52.1484 19.311,Poland
4,Czechia,3,Very Low (<100),49.836 15.5465,Czechia


In [3]:
# Assign lat and lng columns
mmr_global_df['latitude'] = ""
mmr_global_df['longitude'] = ""

for index in mmr_global_df.index:
    location = mmr_global_df.loc[index, 'location'].split(" ")
    lat = location[0]
    lng = location[1]
    mmr_global_df.loc[index, 'latitude'] = lat
    mmr_global_df.loc[index, 'longitude'] = lng

mmr_global_df.head()       

,name,mmr,category,location,country,latitude,longitude
0,Belarus,2,Very Low (<100),53.744 27.947,Belarus,53.744,27.947
1,Italy,2,Very Low (<100),45.3889 10.7776,Italy,45.3889,10.7776
2,Norway,2,Very Low (<100),60.5366 7.4496,Norway,60.5366,7.4496
3,Poland,2,Very Low (<100),52.1484 19.311,Poland,52.1484,19.311
4,Czechia,3,Very Low (<100),49.836 15.5465,Czechia,49.836,15.5465


In [4]:
# Column selection
mmr_global_df_cleaned_df = mmr_global_df[['name', 'mmr', 'category', 'latitude', 'longitude']].copy()
mmr_global_df_cleaned_df.index.name = 'id'

# Display dataframe
mmr_global_df_cleaned_df.head()

,name,mmr,category,latitude,longitude
id,,,,,
0,Belarus,2,Very Low (<100),53.744,27.947
1,Italy,2,Very Low (<100),45.3889,10.7776
2,Norway,2,Very Low (<100),60.5366,7.4496
3,Poland,2,Very Low (<100),52.1484,19.311
4,Czechia,3,Very Low (<100),49.836,15.5465


In [5]:
# csv filepath
global_csv = '../output_file/mmr_global.csv'

mmr_global_df_cleaned_df.to_csv(global_csv, index=False)

### Connect to PostgresSQL database

In [6]:
# Sets an object to utilize the default declarative base in SQL Alchemy
Base = declarative_base()

In [7]:
# Create table schema
class Global(Base):
    __tablename__ = 'mmr_global'
    name = Column(String(255), primary_key=True)
    mmr = Column(Integer)
    category = Column(String)
    latitude = Column(Integer)
    longitude = Column(Integer)

In [8]:
# Create Engine for mmr_global
connect_string = f'postgresql://{username}:{password}@{host}/{database}'
engine = create_engine(connect_string)

In [9]:
# Create (if not already in existence) the tables associated with our classes.
Base.metadata.create_all(engine)

In [10]:
# Store data to SQL database
mmr_global_df_cleaned_df.to_sql(name='mmr_global', con=engine, if_exists='replace', index=False)

In [19]:
# Check of the table exists in database
pd.read_sql_query('select * from mmr_global', con=engine).head()

,name,mmr,category,latitude,longitude
0,Belarus,2,Very Low (<100),53.744,27.947
1,Italy,2,Very Low (<100),45.3889,10.7776
2,Norway,2,Very Low (<100),60.5366,7.4496
3,Poland,2,Very Low (<100),52.1484,19.311
4,Czechia,3,Very Low (<100),49.836,15.5465


### Connect to SQLite local file

In [29]:
# Creates a connection to our DB
database_localpath = 'sqlite:///maternal_mortality.sqlite'
local_engine = create_engine(database_localpath)
conn = local_engine.connect()

In [30]:
# Create (if not already in existence) the tables associated with our classes.
Base.metadata.create_all(local_engine)